<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/week3/part2/02_Wikipedia_NASA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook: Wikipedia + NASA con LangChain 1.0 y Gemini

## 0. Instalación de librerías

In [ ]:
!pip install -qU langchain langchain-google-genai langchain-community wikipedia beautifulsoup4 google-generativeai feedparser

## 1. Importar librerías e instancia de modelo Gemini

In [ ]:
from google.colab import userdata
import os

# Obtener la API key desde userdata
api_key = userdata.get('GOOGLE_API_KEY')

# Opcional: Guardarla como variable de entorno
os.environ['GOOGLE_API_KEY'] = api_key

# Verificar que se haya cargado correctamente
print("API Key cargada:", "Sí" if api_key else "No")

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

chat = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash-lite",
    temperature=0.2
)

print("Modelo listo")

API Key cargada: Sí
Modelo listo


## 2. Integración con Wikipedia

In [ ]:
from langchain_community.document_loaders import WikipediaLoader

def responder_wikipedia(persona, pregunta_arg):
    docs = WikipediaLoader(query=persona, lang="es", load_max_docs=10)
    contexto_extra = docs.load()[0].page_content

    human_prompt = HumanMessagePromptTemplate.from_template(
        "Responde a la siguiente pregunta:\n{pregunta}\n\nAquí tienes contenido adicional:\n{contenido}"
    )

    chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

    solicitud = chat_prompt.format_prompt(
        pregunta=pregunta_arg,
        contenido=contexto_extra
    ).to_messages()

    result = chat.invoke(solicitud)
    print(result.content)

# Ejemplo
responder_wikipedia("Fernando Alonso", "¿Cuándo nació?")

Fernando Alonso nació el **29 de julio de 1981**.


## 3. Integración con la página de noticias de la NASA

In [ ]:
import feedparser

def cargar_rss(url, n=5): # n, el número máximo de noticias a extraer (por defecto 5)
    feed = feedparser.parse(url) # descarga el XML y lo convierte en un objeto Python estructurado:feed.entries contiene la lista de noticias
    items = feed.entries[:n] # solo toma las n (5) mas recientes
    textos = [] # Aquí se almacenarán los textos ya formateados.
    for it in items:
        titulo = getattr(it, "title", "") # Ejemplo: "Obtén el atributo (title) si existe; si no existe, devuelve un valor por defecto"
        resumen = getattr(it, "summary", "")
        link = getattr(it, "link", "")
        textos.append(f"Título: {titulo}\nResumen: {resumen}\nLink: {link}")
    return textos

RSS_NASA = "https://www.nasa.gov/rss/dyn/breaking_news.rss"

posts = cargar_rss(RSS_NASA, n=5)
contenido = "\n\n---\n\n".join(posts)

print(contenido[:800], "...") # Mostrar solo los primeros 800 caracteres


Título: Artemis II Crew Trains on T-38
Resumen: NASA astronaut Christina Koch and Canadian Space Agency astronaut Jeremy Hansen take off on a T-38 training flight from Ellington Field on Feb. 11, 2026, as a waning crescent Moon hovers above. Koch and Hansen, along with NASA astronauts Reid Wiseman and Victor Glover, are part of NASA’s Artemis II mission, the first crewed flight [&#8230;]
Link: https://www.nasa.gov/image-article/artemis-ii-crew-trains-on-t-38/

---

Título: Winds Whip Up Fires and Dust on the Southern Plains
Resumen: Dry, gusty conditions spurred fast-growing fires in Oklahoma and Kansas, along with dangerous dust storms across the region.
Link: https://science.nasa.gov/earth/earth-observatory/winds-whip-up-fires-and-dust-on-the-southern-plains/

---

Título: NASA Investiga ...


## 4. Hacer preguntas sobre las noticias encontradas

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

def responder_nasa(pregunta):
  prompt = ChatPromptTemplate.from_template(
      "Noticias recientes (RSS):\n\n{posts}\n\n"
      "Responde basándote SOLO en esta información:\n{pregunta}"
  )
  messages = prompt.format_messages(posts=contenido, pregunta=pregunta) # posts=contenido, asocia el "contenido" del punto 3.
  result = chat.invoke(messages)
  print(result.content)

# Ejemplo
responder_nasa("Según las publicaciones, ¿qué evento natural se observó y en qué regiones ocurrió? Indica también en qué mes y año sucedió")

Según las publicaciones, se observaron **vientos fuertes que provocaron incendios y tormentas de polvo** en las regiones de **Oklahoma y Kansas**.

No se especifica el mes y año en que sucedió este evento natural.
